In [95]:
#import하기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Ridge

In [96]:
#데이터 준비하기 
train_data = pd.read_csv("E:\데이콘\\전복나이 예측\\데이터\\train.csv")
print(train_data.columns)
train_max_input = train_data[['Gender','Lenght','Diameter','Height','Whole Weight','Shucked Weight','Viscra Weight','Shell Weight']]
train_max_target = train_data['Target'].to_numpy()

ques_data = pd.read_csv("E:\데이콘\\전복나이 예측\\데이터\\test.csv")
ques_input = ques_data[['Gender','Lenght','Diameter','Height','Whole Weight','Shucked Weight','Viscra Weight','Shell Weight']]


Index(['id', 'Gender', 'Lenght', 'Diameter', 'Height', 'Whole Weight',
       'Shucked Weight', 'Viscra Weight', 'Shell Weight', 'Target'],
      dtype='object')


In [97]:
#성별 원핫 인코딩
train_max_input = pd.get_dummies(train_max_input)
ques_input =pd.get_dummies(ques_input)

In [98]:
#훈련세트와 테스트 세트 나누기
train_input, test_input, train_target, test_target = train_test_split(train_max_input,train_max_target, test_size=0.2, random_state=42)

In [99]:
#다루기 쉽게 데이터프레임 ㅡ> 넘파이 배열
train_input = train_input.to_numpy()
test_input = test_input.to_numpy()

In [100]:
#특성공학으로 특성 늘리기
poly = PolynomialFeatures(degree=2,include_bias= False)
poly.fit(train_input)
train_poly=poly.transform(train_input)
test_poly = poly.transform(test_input)
ques_input = ques_input.to_numpy()
train_max_poly = poly.transform(train_max_input)
ques_poly = poly.transform(ques_input)

C:\Users\dlehd\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but PolynomialFeatures was fitted without feature names
  warnings.warn(


In [101]:
#규제 전 전처리 과정
from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
train_max_scaled = ss.transform(train_max_poly)
ques_scaled = ss.transform(ques_poly)
test_scaled = ss.transform(test_poly)

In [102]:
from sklearn.model_selection import GridSearchCV
params = {'alpha':[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000]}
gs = GridSearchCV(Ridge(),params,n_jobs=-1)
gs.fit(train_scaled,train_target)
dt = gs.best_estimator_

In [103]:
print(gs.best_params_)

{'alpha': 1}


In [104]:
import numpy as np
print(np.max(gs.cv_results_['mean_test_score']))

0.5260093388939837


In [105]:

print(dt.score(train_scaled,train_target))
print(dt.score(test_scaled,test_target))

0.5699859993839212
0.5463294658184286


In [106]:
ans=dt.predict(ques_scaled)

In [107]:
##파일에 저장
#f= open("ridge_GS_around_submission.csv","w")
#id = range(1,2925)
#f.write('id, Target \n')
#for i in id:
#    f.write(str(i) + ',' + str(ans[i-1]) + '\n')
#f.close()

In [108]:
#submission = pd.read_csv("E:\데이콘\전복나이 예측\데이터\sample_submission.csv")
#submission['Target'] = ans
#submission.to_csv("Submit_2.csv",index=False)

In [109]:
#예상 평가 점수 만들어보기
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score
pred = dt.predict(test_scaled)
print(pred[:10])
print(test_target[:10])

print(NMAE(test_target,np.around(pred)))

[11.52677965 11.66812986  9.92066909  8.97639876 15.36026928 10.7410434
  8.9418838   9.97683223  7.87296714 13.80477706]
[ 8 13 13  9 13 14 10  9  6 13]
0.14635140644109257
